In [11]:
# import pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# import other libraries
import tqdm
import matplotlib.pyplot as plt
import numpy as np

# import specific functions
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from torchvision import datasets, transforms

# import custom functions
from interpolate import interpolate

In [12]:
# import celeb_a dataset and show example images

data = datasets.CelebA(root='data', split='train', download=True, transform=transforms.ToTensor())

# show example images

fig, ax = plt.subplots(1, 5, figsize=(15, 3))
for i in range(5):
    ax[i].imshow(data[i][0].permute(1, 2, 0))
    ax[i].axis('off')

RuntimeError: The daily quota of the file img_align_celeba.zip is exceeded and it can't be downloaded. This is a limitation of Google Drive and can only be overcome by trying again later.

In [ ]:
# implement convolutional variational autoencoder appropriate for the celeba dataset

class VAE(nn.Module):
    def __init__(self, BOTTLENECK_SIZE=64):
        super(VAE, self).__init__()

        self.bottleneck = BOTTLENECK_SIZE

        # construct en- and decoder
        self.encoder = self.construct_encoder()
        self.decoder = self.construct_decoder()

        #define loss function and optimizer
        self.mse_loss_function = nn.MSELoss(reduction='mean')
        self.mse_weight = 1000

        # define optimizer
        self.optimizer = optim.Adam(self.parameters(), lr=1e-3)

    def loss_function(self, x, x_hat, mu, log_var):
        mse_loss = self.mse_loss_function(x_hat, x)
        kl_loss = torch.mean(-0.5 * sum(1 + log_var - torch.square(mu) - torch.square(torch.exp(log_var))))
        return mse_loss + kl_loss

    def construct_encoder(self):
        encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 256, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 2 * self.bottleneck)
        )
        return encoder
    
    def reparameterise(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def construct_decoder(self):
        decoder = nn.Sequential(
            nn.Linear(self.bottleneck, 256),
            nn.ReLU(),
            nn.Unflatten(1, (256, 1, 1)),
            nn.ConvTranspose2d(256, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose3d(32, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )
        return decoder
    
    def forward(self, x):
        # encode
        x = self.encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=1)
        z = self.reparameterise(mu, logvar)

        # decode
        x = self.decoder(z)
        return x, mu, logvar


In [ ]:
# define training loop

def train_vae(model, X_train, X_val, n_epochs=10, batch_size=32, verbose=True):
    loss_history = {
        'train': [],
        'val': []
    }

    if verbose:
        pbar = tqdm.tqdm(range(n_epochs))
    else:
        pbar = range(n_epochs)
    
    for epoch in pbar:
        # training
        model.train()
        train_loss = 0
        for i in tqdm.tqdm(range(0, X_train.shape[0], batch_size)):
            batch = X_train[i:i+batch_size]
            model.optimizer.zero_grad()
            output, mu, log_var = model(batch)
            loss = model.loss_function(batch, output, mu, log_var)
            loss.backward()
            model.optimizer.step()
            train_loss += loss.item()
        train_loss /= X_train.shape[0]
        loss_history['train'].append(train_loss)

        # validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for i in range(0, X_val.shape[0], batch_size):
                batch = X_val[i:i+batch_size]
                output, mu, log_var = model(batch)
                loss = model.loss_function(batch, output, mu, log_var)
                val_loss += loss.item()
            val_loss /= X_val.shape[0]
            loss_history['val'].append(val_loss)

        #print loss
        if verbose:
            pbar.set_description('Epoch: {}/{}, train loss: {:.4f}, val loss: {:.4f}'.format(epoch+1, n_epochs, train_loss, val_loss))
        
    return loss_history

In [ ]:
# define function to plot loss history

def plot_loss_history(loss_history):
    fig, ax = plt.subplots(1, 1, figsize=(10, 5))

    epochs_linspace = np.linspace(1, len(loss_history['train']), len(loss_history['train']))

    ax.scatter(epochs_linspace, loss_history['train'], label='train loss', marker='o')
    ax.scatter(epochs_linspace, loss_history['val'], label='val loss', marker='x')

    ax.plot(epochs_linspace, loss_history['train'])
    ax.plot(epochs_linspace, loss_history['val'])

    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')

    #ax.set_yscale('log')

    ax.grid(alpha=0.35)
    ax.legend()